In [1]:
import torch
import numpy as np
import time
import sys
from torch.optim.lr_scheduler import StepLR
from os.path import abspath, dirname, exists
sys.path.append(abspath(dirname('demo')))
from module import MADE, TraDE_binary, NADE,  Errormodel, mod2, Loading_code, read_code, Abstractcode

In [2]:
def forward(n_s, m, van, syndrome, device, dtype, k=1):
    condition = syndrome*2-1
    x = (van.partial_forward(n_s=n_s, condition=condition, device=device, dtype=dtype, k=k) + 1)/2
    # print(x)
    x = x[:, m:m+2*k]
    # print(x)
    return x

In [7]:
'''Code para'''
c_type = 'rep'
n, d, k, seed = 7 , 7, 1, 0
info = read_code(d=d, k=k, n=n, seed=seed, c_type=c_type)
Code = Loading_code(info)


device = 'cuda:4'
dtype = torch.float32
m2 = mod2(device=device, dtype=dtype)

'''Errormodel'''
er = 0.1
E = Errormodel(er, e_model='dep')

'''Draw samples'''
trials = 10000
errors = E.generate_error(Code.n,  m=trials, seed=seed)
syndrome = m2.commute(errors, Code.g_stabilizer)
pe = E.pure(Code.pure_es, syndrome, device=device, dtype=dtype)
logical_error = m2.commute(errors, Code.logical_opt)
# print(logical_error)

# Training

In [8]:
'''net para'''
epoch, batch, lr = 20000, 10000, 0.001
ni = Code.m+2*k
depth, width = 4, 20

van = MADE(n=ni, depth=depth, width=width, residual=False).to(device).to(dtype)

optimizer = torch.optim.Adam(van.parameters(), lr=lr)#, momentum=0.9)
scheduler = StepLR(optimizer, step_size=2000, gamma=0.9)
# loss_his = []
# lo_his = []
for l in range(epoch):
    ers = E.generate_error(Code.n, m=batch, seed=False)

    configs = torch.hstack([m2.commute(ers, Code.g_stabilizer), m2.commute(ers, Code.logical_opt)]).to(device).to(dtype)*2-1
    # print(configs)
    # E.configs(sta=Code.g_stabilizer, log=Code.logical_opt, pe=pe, opts=ers).to(device).to(dtype)
    # print(configs[10, Code.m:ni])
    logp = van.log_prob(configs) 
    loss = torch.mean((-logp), dim=0)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if optimizer.state_dict()['param_groups'][0]['lr'] > 0.0002 :
        scheduler.step()
    if (l+1)%1000==0:
        print('Loss:', loss)
    if (l+1) % 10000 == 0:
        lconf = forward(n_s=trials, m=Code.m, van=van, syndrome=syndrome, device=device, dtype=dtype, k=k)
        # print(lconf)

        '''correction'''
        # L = m2.confs_to_opt(confs=lconf, gs=Code.logical_opt)
        # recover = m2.opt_prod(pe, L)
        # # print(m2.commute(recover, Code.logical_opt))
        # check = m2.opt_prod(recover, errors)
        # commute = m2.commute(check, Code.logical_opt)
        # # print(commute)
        # fail = torch.count_nonzero(commute.sum(1))

        fail = torch.count_nonzero((lconf-logical_error).sum(1))

        logical_error_rate = fail/trials
        print(l, 'LER:', logical_error_rate)

Loss: tensor(2.3807, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3538, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3405, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3612, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3504, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3620, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3703, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3736, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3574, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3721, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
9999 LER: tensor(0.3210, device='cuda:4')
Loss: tensor(2.3654, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(2.3382, device='cuda:

# MWPM

In [9]:
from pymatching import Matching

def mwpm(Code, errors, er):
    pcm = Code.PCM.cpu().numpy()
    l1 = m2.rep(Code.logical_opt).int().cpu().numpy()
    l = np.zeros_like(l1)
    l[:, :n], l[:, n:] = l1[:, n:], l1[:, :n]

    weights = np.ones(pcm.shape[1])*np.log((1-er)/er)
    Decoder = Matching(pcm, weights=weights)
    syndrome = m2.commute(errors, Code.g_stabilizer)
    syndrome = syndrome.cpu().numpy()

    error = m2.rep(errors).squeeze().int().cpu().numpy()
    
    correct_number = 0
    t = 0
    for j in range(trials):
        e = error[j]
        s = syndrome[j]

        t1 = time.time()
        #print(s)
        recover = Decoder.decode(s)
        check = (e + recover)%2
        lc = np.sum((check @ l.T) %2)
        t2 = time.time()
        t = t+(t2-t1)
        if lc == 0:
            correct_number+=1
        
    lorate = 1 - correct_number/trials
    return lorate

lorate = mwpm(Code, errors, er)
print(lorate)

0.3165


# Comparing MLD and MWPM

In [ ]:
from module import exact_config
from pymatching import Matching

def trancation(tensor, factor):
    return torch.floor(tensor*factor)/factor

def mwpm(Code, errors, er):
    pcm = Code.PCM.cpu().numpy()
    l1 = m2.rep(Code.logical_opt).int().cpu().numpy()
    l = np.zeros_like(l1)
    l[:, :n], l[:, n:] = l1[:, n:], l1[:, :n]

    weights = np.ones(pcm.shape[1])*np.log((1-er)/er)
    Decoder = Matching(pcm, weights=weights)
    syndrome = m2.commute(errors, Code.g_stabilizer)
    syndrome = syndrome.cpu().numpy()
    error = m2.rep(errors).squeeze().int().cpu().numpy()

    lcs = np.zeros(error.shape[0])
    for j in range(error.shape[0]):
        e = error[j]
        s = syndrome[j]
        recover = Decoder.decode(s)
        # print(e)
        # print(recover)
        check = (e + recover)%2
        # print(recover, e)
        lc = np.sum((check @ l.T) %2)
        if lc != 0:
            lcs[j] = 1
    return lcs   


c_type = 'rep'
n, d, k, seed = 9, 9, 1, 0
er = 0.2
info = read_code(d=d, k=k, n=n, seed=seed, c_type=c_type)
Code = Loading_code(info)

device = 'cuda:4'
dtype = torch.float64
m2 = mod2(device=device, dtype=dtype)



configs = exact_config(D=d-1, N=2**(d-1))
stas = m2.confs_to_opt(confs=configs, gs = Code.g_stabilizer)

trials = 1000
E=Errormodel(e_rate=er, e_model='dep')
errors = E.generate_error(n, trials)#torch.tensor([[0., 0., 0., 0., 0., 1., 0.],[0., 0., 2., 0., 0., 0., 0.]], device='cuda:4', dtype=torch.float32)#
syndrome = m2.commute(errors, Code.g_stabilizer)
logical_error = m2.commute(errors, Code.logical_opt)
# print(logical_error)
logical_error = 2*logical_error[:,0]+logical_error[:,1]
Code.logical_opt = Code.logical_opt.to(device).to(dtype)


lcsmw = mwpm(Code, errors, er)
print(lcsmw.sum()/trials)

non_zero = 0
lcs = np.zeros(trials)

for i in range(trials):
    pe = m2.confs_to_opt(syndrome[i], Code.pure_es)
    log_pcoset = torch.sum(E.log_probability(m2.opt_prod(pe, stas), device=device, dtype=dtype))
    # log_pcosete = torch.sum(E.log_probability(m2.opt_prod(errors[i], stas), device=device, dtype=dtype))

    log_pcoset_x = torch.sum(E.log_probability(m2.opt_prod(m2.opt_prod(pe, Code.logical_opt[0]), stas), device=device, dtype=dtype))
    
    if log_pcoset-log_pcoset_x >= 1e-8:
        non_zero+=1
    log_pcoset_z = torch.sum(E.log_probability(m2.opt_prod(m2.opt_prod(pe, Code.logical_opt[1]), stas), device=device, dtype=dtype))
    log_pcoset_xz = torch.sum(E.log_probability(m2.opt_prod(m2.opts_prod(torch.vstack([pe, Code.logical_opt])), stas), device=device, dtype=dtype))
    # print(log_pcoset.cpu().item(), log_pcoset_x.cpu().item(), log_pcoset_z.cpu().item(), log_pcoset_xz.cpu().item())
    # print(errors[i])
    # print(pe)
    # print(m2.opt_prod(errors[i], Code.logical_opt[0]))
    # print(m2.opt_prod(errors[i], Code.logical_opt[1]))
    # print(m2.opts_prod(torch.vstack([errors[i], Code.logical_opt])))

    # print(log_pcoset, log_pcoset_z)
    # print(torch.argmax(torch.tensor([log_pcoset.item(), log_pcoset_x.item(), log_pcoset_z.item(), log_pcoset_xz.item()])))
    lconf_mld =  torch.argmax(trancation(torch.tensor([log_pcoset.item(), log_pcoset_x.item(), log_pcoset_z.item(), log_pcoset_xz.item()]), 10**10))
    if lconf_mld.item() - logical_error[i].cpu().item() != 0 :
        lcs[i] = 1

# print()s
print('Coset_p_difference:', non_zero)
print('Diff_MLD_MWPM:', abs(lcs-lcsmw).sum()/trials)

    






0.451
Coset_p_difference: 0
Diff_MLD_MWPM: 0.0


In [15]:
print(stas.shape)

torch.Size([256, 9])


In [129]:
pe = torch.tensor([0, 0, 0], device=device, dtype=dtype)
log_pcoset = torch.sum(E.log_probability(m2.opt_prod(pe, stas), device=device, dtype=dtype))
log_pcoset_x = torch.sum(E.log_probability(m2.opt_prod(m2.opt_prod(pe, Code.logical_opt[0]), stas), device=device, dtype=dtype))
print(Code.logical_opt[0])
print(E.log_probability(m2.opt_prod(pe, stas), device=device, dtype=dtype))
print(E.log_probability(m2.opt_prod(m2.opt_prod(pe, Code.logical_opt[0]), stas), device=device, dtype=dtype))
print(log_pcoset, log_pcoset_x)

tensor([1., 1., 1.], device='cuda:4', dtype=torch.float32)
tensor([-0.6694, -5.6392, -5.6392, -5.6392], device='cuda:4',
       dtype=torch.float32)
tensor([-8.1242, -3.1543, -3.1543, -3.1543], device='cuda:4',
       dtype=torch.float32)
tensor(-17.5872, device='cuda:4', dtype=torch.float32) tensor(-17.5872, device='cuda:4', dtype=torch.float32)


# Circuit Level Noise

In [2]:
import stim
from pymatching import Matching

def count_logical_errors(detector_error_model, detection_events, observable_flips):
    # Sample the circuit.
    num_shots = detection_events.shape[0]
    matcher = Matching.from_detector_error_model(detector_error_model)

    # Run the decoder.
    predictions = matcher.decode_batch(detection_events).squeeze()
    # Count the mistakes.
    num_errors = 0
    for shot in range(num_shots):
        actual_for_shot = observable_flips[shot][0]
        predicted_for_shot = predictions[shot]
        # print(actual_for_shot, predicted_for_shot)
        if not np.array_equal(actual_for_shot, predicted_for_shot):
            num_errors += 1
    return num_errors


# Circuit

In [3]:
d, r, k = 17, 1, 1
er = 0.15
trials = 100000

circuit = stim.Circuit.generated(code_task="repetition_code:memory",
                                        distance=d,
                                        rounds=r,
                                        after_clifford_depolarization=er,
                                        before_measure_flip_probability=er,
                                        after_reset_flip_probability=er,
                                        )

dem = circuit.detector_error_model(decompose_errors=False, flatten_loops=True)

sampler = circuit.compile_detector_sampler(seed=None)
dets0, obvs0 = sampler.sample(shots=trials, separate_observables=True)
lmw = count_logical_errors(detector_error_model=dem, detection_events=dets0, observable_flips=obvs0*1)/trials

def forward(n_s, m, van, syndrome, device, dtype, k=1):
    condition = syndrome*2-1
    x = (van.partial_forward(n_s=n_s, condition=condition, device=device, dtype=dtype, k=k) + 1)/2
    x = x[:, m:m+int(2*k)]
    return x

# Training

In [ ]:


epoch = 150000
lr = 0.001
batch  = 20000
dtype = torch.float32
device = 'cuda:3'
ni = len(dets0[0])+len(obvs0[0])
print(ni)
van = MADE(n=ni, depth=3, width=30, residual=False).to(device).to(dtype)

optimizer = torch.optim.Adam(van.parameters(), lr=lr)#, momentum=0.9)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.9)
# loss_his = []
# lo_his = []
for l in range(epoch):
    
    # sampler = defect_circuit.compile_sampler(seed=7453+epoch)
    dets, obvs = sampler.sample(shots=batch, separate_observables=True)

    s = torch.hstack((torch.tensor(dets)*1.0, torch.tensor(obvs)*1.0)).to(device).to(dtype)
    logp = van.log_prob((s*2-1))

    loss = torch.mean((-logp), dim=0)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if optimizer.state_dict()['param_groups'][0]['lr'] > 0.0002 :
        scheduler.step()
    
    if (l) % 1000 == 0:
        print(loss)
        print('epoch:', l)
        t0 = time.time()
        lconf = forward(n_s=trials, m=ni-k, van=van, syndrome=torch.tensor(dets0)*1.0, device=device, dtype=dtype, k=k/2)
        # print(lconf[0])
        aclo = (torch.tensor(obvs0)*1.0).to(device).to(dtype)
        # print(aclo[0])
        logical_error_rate = torch.count_nonzero((aclo-lconf).sum(1))/trials
        t1 = time.time()
        print(t1-t0)
        print('mw:', lmw)
        print('gnd:', logical_error_rate)

path = abspath(dirname('demo'))+'/decoding/net/cir/'+'rep_d{}_r{}_k{}_er{}.pt'.format(d, r, k, er)
print(path)
torch.save(van, path)

33
tensor(22.8778, device='cuda:3', dtype=torch.float32, grad_fn=<MeanBackward1>)
epoch: 0
0.013798952102661133
mw: 0.0611
gnd: tensor(0.2983, device='cuda:3')
tensor(21.7191, device='cuda:3', dtype=torch.float32, grad_fn=<MeanBackward1>)
epoch: 1000
0.006373405456542969
mw: 0.0611
gnd: tensor(0.1200, device='cuda:3')
tensor(21.5431, device='cuda:3', dtype=torch.float32, grad_fn=<MeanBackward1>)
epoch: 2000
0.013817548751831055
mw: 0.0611
gnd: tensor(0.0891, device='cuda:3')
tensor(21.5329, device='cuda:3', dtype=torch.float32, grad_fn=<MeanBackward1>)
epoch: 3000
0.05894351005554199
mw: 0.0611
gnd: tensor(0.0807, device='cuda:3')
tensor(21.4755, device='cuda:3', dtype=torch.float32, grad_fn=<MeanBackward1>)
epoch: 4000
0.005953788757324219
mw: 0.0611
gnd: tensor(0.0735, device='cuda:3')
tensor(21.4241, device='cuda:3', dtype=torch.float32, grad_fn=<MeanBackward1>)
epoch: 5000
0.005702018737792969
mw: 0.0611
gnd: tensor(0.0693, device='cuda:3')
tensor(21.4422, device='cuda:3', dtype=to

In [3]:
import sys
print(sys.path)
import os
print(os.environ.get("LD_LIBRARY_PATH", "Not set"))

['/data/caohanyan/Aproject/GND', '/data/caohanyan/anaconda3/envs/chy/lib/python39.zip', '/data/caohanyan/anaconda3/envs/chy/lib/python3.9', '/data/caohanyan/anaconda3/envs/chy/lib/python3.9/lib-dynload', '', '/data/caohanyan/anaconda3/envs/chy/lib/python3.9/site-packages', '/data/caohanyan/anaconda3/envs/chy/lib/python3.9/site-packages/setuptools/_vendor']
Not set


In [1]:
from pymatching import Matching

In [11]:
import stim
import numpy as np
import torch
from pymatching import Matching
from os.path import abspath, dirname, exists

def count_logical_errors(detector_error_model, detection_events, observable_flips):
    # Sample the circuit.
    num_shots = detection_events.shape[0]
    matcher = Matching.from_detector_error_model(detector_error_model)

    # Run the decoder.
    predictions = matcher.decode_batch(detection_events).squeeze()
    # Count the mistakes.
    num_errors = 0
    for shot in range(num_shots):
        actual_for_shot = observable_flips[shot][0]
        predicted_for_shot = predictions[shot]
        # print(actual_for_shot, predicted_for_shot)
        if not np.array_equal(actual_for_shot, predicted_for_shot):
            num_errors += 1
    return num_errors


def forward(n_s, m, van, syndrome, device, dtype, k=1):
    condition = syndrome*2-1
    x = (van.partial_forward(n_s=n_s, condition=condition, device=device, dtype=dtype, k=k) + 1)/2
    # print(x)
    x = x[:, m:m+int(2*k)]
    # print(x)
    return x

d, r, k = 17, 1, 1
ert = 0.1
trials = 10000
a = 100
dtype = torch.float32
device = 'cuda:3'

path = abspath(dirname('demo'))+'/decoding/net/cir/'+'rep_d{}_r{}_k{}_er{}a.pt'.format(d, r, k, ert)
van = torch.load(path).to(device).to(dtype)

ers = np.linspace(0.1, 0.12, 3)#[0.08]



ls = np.zeros((a, len(ers)))
lmws = np.zeros((a, len(ers)))


for j in range(a):
    for i in range(len(ers)):
        er = ers[i]
        circuit = stim.Circuit.generated(code_task="repetition_code:memory",
                                            distance=d,
                                            rounds=r,
                                            after_clifford_depolarization=er,
                                            before_measure_flip_probability=er,
                                            after_reset_flip_probability=er,
                                            )

        dem = circuit.detector_error_model(decompose_errors=False, flatten_loops=True)

        sampler = circuit.compile_detector_sampler(seed=j)#721*j+int(100*er)
        dets0, obvs0 = sampler.sample(shots=trials, separate_observables=True)
        lmw = count_logical_errors(detector_error_model=dem, detection_events=dets0, observable_flips=obvs0*1)/trials
        lmws[j, i] = lmw

        lconf = forward(n_s=trials, m=(d-1)*(r+1), van=van, syndrome=torch.tensor(dets0)*1.0, device=device, dtype=dtype, k=k/2)
        # print(lconf[0])
        aclo = (torch.tensor(obvs0)*1.0).to(device).to(dtype)
        # print(aclo[0])
        logical_error_rate = torch.count_nonzero((aclo-lconf).sum(1))/trials
        ls[j, i] = logical_error_rate.item()
        # print(lmw, logical_error_rate.item())


gnd = ls.mean(0)
gndstd = ls.std(0)
mw = lmws.mean(0)
mwstd = lmws.std(0)
print(ls.mean(0), ls.std(0))
print(lmws.mean(0), lmws.std(0))
print('relative improvement:', (mw-gnd)/mw)

[0.00704  0.01211  0.019561] [0.0009117  0.00107773 0.00141936]
[0.007618 0.013039 0.021795] [0.00097122 0.0011341  0.00140651]
relative improvement: [0.07587293 0.0712478  0.10250057]


In [10]:
print('relative improvement:', (mw-gnd)/mw)

relative improvement: [0.07587293 0.0712478  0.10250057]


In [8]:
import stim
import numpy as np
import torch
from pymatching import Matching
from os.path import abspath, dirname, exists

def count_logical_errors(detector_error_model, detection_events, observable_flips):
    # Sample the circuit.
    num_shots = detection_events.shape[0]
    matcher = Matching.from_detector_error_model(detector_error_model)

    # Run the decoder.
    predictions = matcher.decode_batch(detection_events).squeeze()
    # Count the mistakes.
    num_errors = 0
    for shot in range(num_shots):
        actual_for_shot = observable_flips[shot][0]
        predicted_for_shot = predictions[shot]
        # print(actual_for_shot, predicted_for_shot)
        if not np.array_equal(actual_for_shot, predicted_for_shot):
            num_errors += 1
    return num_errors


def forward(n_s, m, van, syndrome, device, dtype, k=1):
    condition = syndrome*2-1
    x = (van.partial_forward(n_s=n_s, condition=condition, device=device, dtype=dtype, k=k) + 1)/2
    # print(x)
    x = x[:, m:m+int(2*k)]
    # print(x)
    return x

d, r, k = 17, 1, 1
ert = 0.1
trials = 10000
a = 100
dtype = torch.float32
device = 'cuda:3'

path = abspath(dirname('demo'))+'/decoding/net/cir/'+'rep_d{}_r{}_k{}_er{}a.pt'.format(d, r, k, ert)
van = torch.load(path).to(device).to(dtype)

ers = np.linspace(0.1, 0.12, 3)#[0.08]



ls = np.zeros((a, len(ers)))
lmws = np.zeros((a, len(ers)))


for j in range(a):
    for i in range(len(ers)):
        er = ers[i]
        circuit = stim.Circuit.generated(code_task="repetition_code:memory",
                                            distance=d,
                                            rounds=r,
                                            after_clifford_depolarization=er,
                                            before_measure_flip_probability=er,
                                            after_reset_flip_probability=er,
                                            )

        dem = circuit.detector_error_model(decompose_errors=False, flatten_loops=True)

        sampler = circuit.compile_detector_sampler(seed=0)#721*j+int(100*er)
        dets0, obvs0 = sampler.sample(shots=trials, separate_observables=True)
        lmw = count_logical_errors(detector_error_model=dem, detection_events=dets0, observable_flips=obvs0*1)/trials
        lmws[j, i] = lmw

        lconf = forward(n_s=trials, m=(d-1)*(r+1), van=van, syndrome=torch.tensor(dets0)*1.0, device=device, dtype=dtype, k=k/2)
        # print(lconf[0])
        aclo = (torch.tensor(obvs0)*1.0).to(device).to(dtype)
        # print(aclo[0])
        logical_error_rate = torch.count_nonzero((aclo-lconf).sum(1))/trials
        ls[j, i] = logical_error_rate.item()
        # print(lmw, logical_error_rate.item())


gnd = ls.mean(0)
gndstd = ls.std(0)
mw = lmws.mean(0)
mwstd = lmws.std(0)
print(ls.mean(0), ls.std(0))
print(lmws.mean(0), lmws.std(0))
print('relative improvement:', (mw-gnd)/mw)

[0.0081 0.01   0.0203] [1.73472348e-18 6.93889390e-18 6.93889390e-18]
[0.0093 0.0118 0.022 ] [1.73472348e-17 1.56125113e-17 6.93889390e-18]
relative improvement: [0.12903226 0.15254237 0.07727273]
